In [1]:
import time
from datetime import datetime
from os import mkdir
from os.path import exists, join

import pandas as pd
import numpy as np
import scipy as sp

import psutil

In [2]:
if not exists('logs'):
    mkdir(log_dir)

In [3]:
now = datetime.now()
log_dir = join('logs', now.strftime('%d-%H'))
if not exists(log_dir):
    mkdir(log_dir)
log_dir

'logs/31-21'

In [4]:
timespans = [str(x.date())[-2:] for x in pd.date_range('2017/6/5', '2017/6/11')]

In [5]:
worker_dirs = timespans + ['test']

In [6]:
feture_extract_path = ['extract_hotel_feature', 'extract_basic_room_feature',
                      'extract_hotel_room_feature', 'extract_room_feature',
                      'extract_uid_feture', 'extract_order_feature']

In [48]:
# feture_extract_path = ['extract_uid_feture']

In [18]:
feture_combine_path = ['combine_sample']

In [19]:
max_run = 4

In [22]:
def get_args(feture_extract_path, worker_dirs):
    for p in feture_extract_path:
        for d in worker_dirs:
            yield [p, d]

In [40]:
def check(p):
    pm = p.memory_info()
    is_zero = all(map(lambda x:getattr(pm, x)==0, ['rss','vms', 'shared', 'text', 'lib', 'data', 'dirty']))
    return is_zero

In [41]:
def is_wait(pool, max_pool=4):
    if len(pool) < max_pool:
        return False
    else:
        time.sleep(30)
        not_run_p = []
        for i, p in enumerate(pool[:]):
            not_run = check(p)
            print(datetime.now(), 'process', i, 'not runing' if not_run else 'run')
            if not_run:
                not_run_p.append(i)
        not_run_p.sort(reverse=True)
        for i in not_run_p:
            p = pool.pop(i)
            p.kill()
        return True

In [42]:
def run_args(extract_args, max_pool=4):
    pool = []
    for p, n in extract_args:
        print(datetime.now(), 'begin', p, n)
        fn = join(log_dir, '{}_{}.txt'.format(p, n))
        pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
        while is_wait(pool, max_pool):
            print(datetime.now(), 'run in ', p, n)

In [28]:
# for p, n in extract_args:
#     print(datetime.now(), 'begin', p, n)
#     fn = join(log_dir, '{}_{}.txt'.format(p, n))
#     pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
#     while is_wait(pool):
#         print(datetime.now(), 'run in ', p, n)

In [45]:
extract_args = get_args(feture_extract_path, worker_dirs)
combine_args = get_args(feture_combine_path, worker_dirs)

In [46]:
run_args(extract_args, max_run)

2017-07-31 22:36:58.820711 begin extract_uid_feture 05
2017-07-31 22:36:58.835152 begin extract_uid_feture 06
2017-07-31 22:36:58.844220 begin extract_uid_feture 07
2017-07-31 22:36:58.853939 begin extract_uid_feture 08
2017-07-31 22:37:28.894839 process 0 run
2017-07-31 22:37:28.895091 process 1 run
2017-07-31 22:37:28.895238 process 2 run
2017-07-31 22:37:28.895379 process 3 run
2017-07-31 22:37:28.895443 run in  extract_uid_feture 08
2017-07-31 22:37:58.925609 process 0 run
2017-07-31 22:37:58.925837 process 1 run
2017-07-31 22:37:58.925987 process 2 not runing
2017-07-31 22:37:58.926130 process 3 run
2017-07-31 22:37:58.926503 run in  extract_uid_feture 08
2017-07-31 22:37:58.926571 begin extract_uid_feture 09
2017-07-31 22:38:28.962794 process 0 not runing
2017-07-31 22:38:28.963231 process 1 not runing
2017-07-31 22:38:28.963331 process 2 not runing
2017-07-31 22:38:28.963407 process 3 run
2017-07-31 22:38:28.963891 run in  extract_uid_feture 09
2017-07-31 22:38:28.963930 begin e

In [47]:
run_args(combine_args, max_run)

2017-07-31 22:38:59.025224 begin combine_sample 05
2017-07-31 22:38:59.053224 begin combine_sample 06
2017-07-31 22:38:59.069343 begin combine_sample 07
2017-07-31 22:38:59.097851 begin combine_sample 08
2017-07-31 22:39:29.145021 process 0 not runing
2017-07-31 22:39:29.145600 process 1 not runing
2017-07-31 22:39:29.145736 process 2 not runing
2017-07-31 22:39:29.145903 process 3 not runing
2017-07-31 22:39:29.146761 run in  combine_sample 08
2017-07-31 22:39:29.146824 begin combine_sample 09
2017-07-31 22:39:29.159262 begin combine_sample 10
2017-07-31 22:39:29.164537 begin combine_sample 11
2017-07-31 22:39:29.172568 begin combine_sample test
2017-07-31 22:39:59.225203 process 0 not runing
2017-07-31 22:39:59.238380 process 1 not runing
2017-07-31 22:39:59.238492 process 2 not runing
2017-07-31 22:39:59.238620 process 3 not runing
2017-07-31 22:39:59.251642 run in  combine_sample test
